# Pretrained weights, Transfer learning, freeze weights

Too much time to train. How large is the problem we are dealing with. Number of parameters?

eg: above - 138M !!  
that with 50K images. (probably not enough data for this problem)

two problems - time, not enough data.

use pretrained models.  
pretrained with some data .. have optimized weights - use that and train only some part of the n/w (ie train some of the weights.)

below :    
use **pretrained weights**, except in the last layer in classifier.  
other weights as it is.  
**make their requires_grad = False to 'freeze' them - they won't be updated.**

In [1]:
import numpy as np
import torch

import matplotlib.pyplot as plt

In [2]:
import torchvision
import torchvision.transforms as transforms

In [3]:
import torch.nn as nn
import torch.optim as optim

from torchvision import models

In [4]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### Dataset

In [5]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

# crop - portion of the image instead of shrinking them unequally along differet dim
# crop from a random location.
# here 32x32 to 224x224 - blow up.
# normalize for images also like other data
# mean, SD in 3 channels.
# 0.5,0.5 -> most values b/w 0,1
# ideally we should find the mean, SD(of train data) and then use those values
# it will be available in the dataset website


transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

# here we are using the same set of transforms for both test and train
# but in general we might want different ones.
# eg: data augmentation - (eg. flip horizontally) - in train
# but not in test.

# should use the same mean, SD for normalizing test data also.
# same as that of train data.

###############################################################

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                        download=False, 
                                        transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, 
                                        download=False, 
                                        transform=transform_test)

### Utils

In [6]:
# function to find accuracy - given dataloader, model
def evaluation(dataloader, model):
    model.eval()
    # model be already in device
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        # move data to device (in batches)
        inputs, labels = inputs.to(device), labels.to(device)
        with torch.no_grad():
            outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

# device is global

## Model (Pretrained + Modify last layer)

The default value of model-dir is \\$TORCH_HOME/models where \\$TORCH_HOME defaults to ~/.torch. A different path can be set by setting the env variable TORCH_HOME using os.environ['TORCH_HOME'].


In [11]:
import os
os.environ['TORCH_HOME'] = './torch_home'

Different pretrained-weights can be choosen. See docs.

In [ ]:
vgg = models.vgg16_bn(weights='DEFAULT') #pretrained
# this will LOAD OPTIMIZED WEIGHTS, BIASES (trained on imageNet dataset).

# FREEZING ALL PARAMETERS
# set requires_grad = False for the parameters
for param in vgg.parameters():
    param.requires_grad = False

num_classes=10
    
final_in_features = vgg.classifier[6].in_features

# change the last layer - because are making a new one - for that parameters will be made
# and their req_grad will be true. so they will get updated.

# can change a single thing this way also? not need to replace the entire sequential?

vgg.classifier[6] = nn.Linear(final_in_features, num_classes)

# remake layers - that need to be trained.

so in above case - except last layer everything is static. and the last layer parameters get updated with our dataset.    
training lesser parameter - faster.

usually we train the whole classifier part - using pretrained weights of features part.   
(ie, using the representation - learn classifier from that.)

In [13]:
# check

for param in vgg.parameters():
    if param.requires_grad:
        print(param.shape)

# only they have req_grad = True.

torch.Size([10, 4096])
torch.Size([10])


## Train

In [ ]:
# dataloader
batch_size = 16
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

# build model
vgg = vgg.to(device)

# loss function
loss_fn = nn.CrossEntropyLoss()

# build opt
opt = optim.SGD(vgg.parameters(), lr=0.05)

#########################################################################################

loss_per_epoch_arr = []
num_epochs = 1

# number of batches in dataset
num_steps = np.ceil(50000/batch_size)

for epoch in range(num_epochs):

    for i, data in enumerate(trainloader, 0):
        
        # data
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad() # reset gradients
        
        # forward pass
        outputs = vgg(inputs)
        
        # loss
        loss = loss_fn(outputs, labels)
        
        # compute gradients
        loss.backward()
        
        # update parameters
        opt.step()
        
        # DELETING CURRENT BATCH TO SAVE MEMORY - AS ITS NOT NEEDED AGAIN
        del inputs, labels, outputs
        torch.cuda.empty_cache()
        
        # print loss every 100 steps
        if i % 100 == 0:
            print('Step: %d/%d, Loss: %0.2f' % (i, num_steps, loss.item()) )
        
    loss_per_epoch_arr.append(loss.item())
    
    # Evaluation after each epoch    
    # no grad is there in evaluation fucntion. 
    # can add here also to be safe.
    print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (
        epoch, num_epochs, 
        evaluation(testloader, vgg), evaluation(trainloader, vgg)))
    
    
plt.plot(loss_per_epoch_arr)
plt.show()

Step: 0/3125, Loss: 2.33
Step: 100/3125, Loss: 1.48
Step: 200/3125, Loss: 1.93
Step: 300/3125, Loss: 1.78
Step: 400/3125, Loss: 2.18
Step: 500/3125, Loss: 1.89
Step: 600/3125, Loss: 2.11
Step: 700/3125, Loss: 2.60
Step: 800/3125, Loss: 2.06
Step: 900/3125, Loss: 1.52
Step: 1000/3125, Loss: 2.60
Step: 1100/3125, Loss: 1.39
Step: 1200/3125, Loss: 2.46
Step: 1300/3125, Loss: 1.62
Step: 1400/3125, Loss: 2.76
Step: 1500/3125, Loss: 1.77
Step: 1600/3125, Loss: 2.40
Step: 1700/3125, Loss: 2.26
Step: 1800/3125, Loss: 2.39
Step: 1900/3125, Loss: 3.35
Step: 2000/3125, Loss: 2.76
Step: 2100/3125, Loss: 2.73
Step: 2200/3125, Loss: 2.32
Step: 2300/3125, Loss: 2.24
Step: 2400/3125, Loss: 1.70
Step: 2500/3125, Loss: 2.26
Step: 2600/3125, Loss: 1.29
Step: 2700/3125, Loss: 1.94
Step: 2800/3125, Loss: 2.15
Step: 2900/3125, Loss: 1.32
Step: 3000/3125, Loss: 1.50
Step: 3100/3125, Loss: 1.62


having only some trainable parameters - and using the rest from **"transfer learning"** from a different dataset.